In [1]:
!pip install evaluate rouge_score wandb transformers[torch] sentencepiece -q
!pip install accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

## Get Model

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("DylanonWic/mT5_summarize_th_en")

# my_model = AutoModelForSeq2SeqLM.from_pretrained("DylanonWic/mT5_summarize_th_en")
# base_model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
my_model = pipeline("summarization", model="DylanonWic/mT5_summarize_th_en", device="cuda:0")
base_model = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum", device="cuda:0")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Setup evaluation

In [3]:
import nltk
import evaluate
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [4]:
import numpy as np

def compute_metrics(preds, labels):
    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

## Get model

In [5]:
from datasets import load_dataset

dataset_en = load_dataset("csebuetnlp/xlsum", 'english', split="validation")
dataset_th = load_dataset("csebuetnlp/xlsum", 'thai', split="validation")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
dataset = dataset_en[:200]

predictions = [res['summary_text'] for res in my_model(dataset['text'], batch_size=2)]
references = dataset['summary']

results = compute_metrics(preds=predictions, labels=references)
results

{'rouge1': 0.3678517300926827,
 'rouge2': 0.14472990942796587,
 'rougeL': 0.2936710898693531,
 'rougeLsum': 0.29521397453708276}

In [ ]:
dataset = dataset_th[:200]

predictions = [res['summary_text'] for res in my_model(dataset['text'], batch_size=2)]
references = dataset['summary']

results = compute_metrics(preds=predictions, labels=references)
results

In [ ]:
dataset = dataset_en[:200]

predictions = [res['summary_text'] for res in base_model(dataset['text'], batch_size=2)]
references = dataset['summary']

results = compute_metrics(preds=predictions, labels=references)
results

In [ ]:
dataset = dataset_th[:200]

predictions = [res['summary_text'] for res in base_model(dataset['text'], batch_size=2)]
references = dataset['summary']

results = compute_metrics(preds=predictions, labels=references)
results